In [7]:
from mytypes import Queue
import random
import numpy as np
import pandas as pd

In [8]:
# A class to represent a single customer in an M/D/1 queue simulation.
# Each customer has three attributes:
#
#  - cid: A customer identifier (can be anything, but we will use consecutive integers)
#  - arrival_time: The time at which the customer arrived at the queue
#  - departure_time: The time at which the customer departed the queue
class Customer(object):
    CUSTOMER_ID = 0

    def __init__(self, arrival_time):
        Customer.CUSTOMER_ID += 1
        self.cid = Customer.CUSTOMER_ID
        self.arrival_time = arrival_time
        self.departure_time = None
        
    @property
    def wait(self):
        if self.departure_time is None:
            return None
        else:
            return self.departure_time - self.arrival_time
        
    def __str__(self):
        return "Customer({}, {})".format(self.cid, self.arrival_time)
    
    def __repr__(self):
        return str(self)

In [9]:
import random

# simulate_md1: Simulates an M/D/1 queue.
#
# In an M/D/1 queue que have:
#   
# - Arrivals follow a Markov process (M)
# - The time to service each customer is deterministic (D)
# - There is only one server (1)
#
# The function takes three parameters (plus one optional parameter)
#
# - lambd: The simulation uses an exponential distribution to determine
#          the arrival time of the next customer. This parameters is the
#          lambda parameter to an exponential distribution (specifically,
#          Python's random.expovariate)
# - mu: The rate at which customers are serviced. The larger this value is,
#       the more customers will be serviced per unit of time
# - max_time: The maximum time of the simulation
# - verbosity (optional): Can be 0 (no output), 1 (print state of the queue
#                         at each time), or 2 (same as 1, but also print when
#                         each customer arrives and departs)
#
# The function returns two lists: one with all the customers that were served
# during the simulation, and one with all the customers that were yet to be
# served when the simulation ended.
#
def simulate_md1(lambd, mu, max_time, verbosity = 0):
    md1 = Queue()
    L_array = []

    # Our return values: the list of customers that have been
    # served, and the list of customers that haven't been served
    served_customers = []
    unserved_customers = []
    
    # The type of simulation we have implemented in this function
    # is known as a "discrete event simulation"
    # (https://en.wikipedia.org/wiki/Discrete_event_simulation), where
    # we simulate a discrete sequence of events: customer arrivals
    # and customer departures. So, we only need to keep track of when 
    # the next arrival and the next departure will take place (because 
    # nothing interesting happens between those two types of events). 
    # Then, in each step of the simulation, we simply advance the 
    # simulation clock to earliest next event. Note that, because
    # we have a single server, this can be easily done with just
    # two variables.

    next_arrival = random.expovariate(lambd)
    next_service = next_arrival + 1/mu
        
    # We initialize the simulation's time to the earliest event:
    # the next arrival time
    t = next_arrival
    
    while t < max_time:

        # Process a new arrival
        if t == next_arrival:
            customer = Customer(arrival_time = t)
            md1.enqueue(customer)

            if verbosity >= 2:
                print("{:10.2f}: Customer {} arrives".format(t, customer.cid))

            next_arrival = t + random.expovariate(lambd)
            
        # The customer at the head of the queue has been served
        if t == next_service:
            done_customer = md1.dequeue()
            done_customer.departure_time = t
            
            served_customers.append(done_customer)

            if verbosity >= 2:
                print("{:10.2f}: Customer {} departs".format(t, done_customer.cid))            
            
            if md1.is_empty():
                # The next service time will be 1/mu after the next arrival
                next_service = next_arrival + 1/mu
            else:
                # We start serving the next customer, so the next service time
                # will be 1/mu after the current time.
                next_service = t + 1/mu
            
        if verbosity >= 1:
            print("{:10.2f}: {}".format(t, "#"*md1.length))
            L_array.append(md1.length)
            
        # Advance the simulation clock to the next event
        t = min(next_arrival, next_service)
        
    # Any remaining customers in the queue haven't been served
    while not md1.is_empty():
        unserved_customers.append(md1.dequeue())
        
    return L_array

In [10]:
simulate_md1(0.167, 0.15, 100, verbosity=2)

      3.72: Customer 1 arrives
      3.72: #
      7.10: Customer 2 arrives
      7.10: ##
     10.39: Customer 1 departs
     10.39: #
     11.01: Customer 3 arrives
     11.01: ##
     15.21: Customer 4 arrives
     15.21: ###
     17.06: Customer 2 departs
     17.06: ##
     23.72: Customer 3 departs
     23.72: #
     30.39: Customer 4 departs
     30.39: 
     31.07: Customer 5 arrives
     31.07: #
     37.74: Customer 5 departs
     37.74: 
     43.00: Customer 6 arrives
     43.00: #
     44.31: Customer 7 arrives
     44.31: ##
     49.66: Customer 6 departs
     49.66: #
     53.90: Customer 8 arrives
     53.90: ##
     55.55: Customer 9 arrives
     55.55: ###
     56.33: Customer 7 departs
     56.33: ##
     57.22: Customer 10 arrives
     57.22: ###
     57.23: Customer 11 arrives
     57.23: ####
     57.75: Customer 12 arrives
     57.75: #####
     63.00: Customer 8 departs
     63.00: ####
     69.66: Customer 9 departs
     69.66: ###
     73.49: Customer 13 arrive

[1,
 2,
 1,
 2,
 3,
 2,
 1,
 0,
 1,
 0,
 1,
 2,
 1,
 2,
 3,
 2,
 3,
 4,
 5,
 4,
 3,
 4,
 5,
 4,
 3,
 4,
 5,
 6,
 5,
 6,
 7,
 8,
 9,
 10,
 9]

In [15]:
#loop through 9 different cases and record the results
result_df = pd.DataFrame(columns = ['n', 'lambda', 'mean', 'variance'])
j = 0
L_array = []
full_L_array = []


for lambd in [1.1]:
    for n in [1000]:
        L_array = []
        full_L_array = []
        for i in range(10):
            L_array = simulate_md1(lambd, 1, n, verbosity=2)
            #print(L_array)
            #print('current run' + str(i))
            full_L_array.extend(L_array)
            #print(full_L_array)
        mean_L = np.mean(full_L_array)
        var_L = np.var(full_L_array)
        #new_row = pd.DataFrame([str(n), str(lambd), mean_L, var_L])
        rows = [n, lambd, mean_L, var_L]
        result_df.loc[j] = rows
        #result_df.append(new_row)
        print(str(n), str(lambd), mean_L, var_L)
        j+=1
        #df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))

      1.27: Customer 22070 arrives
      1.27: #
      1.39: Customer 22071 arrives
      1.39: ##
      1.41: Customer 22072 arrives
      1.41: ###
      2.27: Customer 22070 departs
      2.27: ##
      3.27: Customer 22071 departs
      3.27: #
      3.55: Customer 22073 arrives
      3.55: ##
      3.74: Customer 22074 arrives
      3.74: ###
      4.27: Customer 22072 departs
      4.27: ##
      5.25: Customer 22075 arrives
      5.25: ###
      5.27: Customer 22073 departs
      5.27: ##
      6.27: Customer 22074 departs
      6.27: #
      7.10: Customer 22076 arrives
      7.10: ##
      7.27: Customer 22075 departs
      7.27: #
      7.95: Customer 22077 arrives
      7.95: ##
      8.27: Customer 22076 departs
      8.27: #
      8.53: Customer 22078 arrives
      8.53: ##
      9.27: Customer 22077 departs
      9.27: #
      9.45: Customer 22079 arrives
      9.45: ##
     10.27: Customer 22078 departs
     10.27: #
     11.27: Customer 22079 departs
     11.27: 
     1

    220.93: Customer 22306 arrives
    220.93: ###################
    221.17: Customer 22307 arrives
    221.17: ####################
    221.63: Customer 22288 departs
    221.63: ###################
    222.07: Customer 22308 arrives
    222.07: ####################
    222.63: Customer 22289 departs
    222.63: ###################
    223.63: Customer 22290 departs
    223.63: ##################
    224.44: Customer 22309 arrives
    224.44: ###################
    224.63: Customer 22291 departs
    224.63: ##################
    225.03: Customer 22310 arrives
    225.03: ###################
    225.39: Customer 22311 arrives
    225.39: ####################
    225.63: Customer 22292 departs
    225.63: ###################
    225.68: Customer 22312 arrives
    225.68: ####################
    226.22: Customer 22313 arrives
    226.22: #####################
    226.63: Customer 22293 departs
    226.63: ####################
    226.97: Customer 22314 arrives
    226.97: ##########

    429.63: Customer 22496 departs
    429.63: ##################################################
    430.41: Customer 22547 arrives
    430.41: ###################################################
    430.63: Customer 22497 departs
    430.63: ##################################################
    430.93: Customer 22548 arrives
    430.93: ###################################################
    431.63: Customer 22498 departs
    431.63: ##################################################
    432.63: Customer 22499 departs
    432.63: #################################################
    432.65: Customer 22549 arrives
    432.65: ##################################################
    433.11: Customer 22550 arrives
    433.11: ###################################################
    433.15: Customer 22551 arrives
    433.15: ####################################################
    433.28: Customer 22552 arrives
    433.28: #####################################################
    433.63: C

    686.13: ##############################################################################################
    686.63: Customer 22753 departs
    686.63: #############################################################################################
    687.63: Customer 22754 departs
    687.63: ############################################################################################
    688.63: Customer 22755 departs
    688.63: ###########################################################################################
    689.23: Customer 22847 arrives
    689.23: ############################################################################################
    689.27: Customer 22848 arrives
    689.27: #############################################################################################
    689.63: Customer 22756 departs
    689.63: ############################################################################################
    689.89: Customer 22849 arrives
    689.89: #####

     26.87: ####
     27.11: Customer 23222 departs
     27.11: ###
     28.11: Customer 23223 departs
     28.11: ##
     29.11: Customer 23224 departs
     29.11: #
     29.79: Customer 23226 arrives
     29.79: ##
     30.11: Customer 23225 departs
     30.11: #
     30.38: Customer 23227 arrives
     30.38: ##
     31.11: Customer 23226 departs
     31.11: #
     32.11: Customer 23227 departs
     32.11: 
     32.45: Customer 23228 arrives
     32.45: #
     32.49: Customer 23229 arrives
     32.49: ##
     33.45: Customer 23228 departs
     33.45: #
     33.75: Customer 23230 arrives
     33.75: ##
     34.40: Customer 23231 arrives
     34.40: ###
     34.45: Customer 23229 departs
     34.45: ##
     34.86: Customer 23232 arrives
     34.86: ###
     35.09: Customer 23233 arrives
     35.09: ####
     35.45: Customer 23230 departs
     35.45: ###
     36.18: Customer 23234 arrives
     36.18: ####
     36.44: Customer 23235 arrives
     36.44: #####
     36.45: Customer 23231 de

...

    219.46: ###################################
    219.85: Customer 31180 departs
    219.85: ##################################
    220.85: Customer 31181 departs
    220.85: #################################
    221.85: Customer 31182 departs
    221.85: ################################
    221.98: Customer 31215 arrives
    221.98: #################################
    222.28: Customer 31216 arrives
    222.28: ##################################
    222.85: Customer 31183 departs
    222.85: #################################
    223.19: Customer 31217 arrives
    223.19: ##################################
    223.85: Customer 31184 departs
    223.85: #################################
    224.85: Customer 31185 departs
    224.85: ################################
    225.85: Customer 31186 departs
    225.85: ###############################
    226.85: Customer 31187 departs
    226.85: ##############################
    226.99: Customer 31218 arrives
    226.99: ##################

    631.82: ##################################################
    631.85: Customer 31592 departs
    631.85: #################################################
    632.85: Customer 31593 departs
    632.85: ################################################
    632.98: Customer 31642 arrives
    632.98: #################################################
    633.17: Customer 31643 arrives
    633.17: ##################################################
    633.38: Customer 31644 arrives
    633.38: ###################################################
    633.85: Customer 31594 departs
    633.85: ##################################################
    634.16: Customer 31645 arrives
    634.16: ###################################################
    634.85: Customer 31595 departs
    634.85: ##################################################
    635.37: Customer 31646 arrives
    635.37: ###################################################
    635.85: Customer 31596 departs
    635.85: #########

     13.12: Customer 32045 departs
     13.12: ##
     13.45: Customer 32048 arrives
     13.45: ###
     14.12: Customer 32046 departs
     14.12: ##
     14.41: Customer 32049 arrives
     14.41: ###
     14.72: Customer 32050 arrives
     14.72: ####
     14.79: Customer 32051 arrives
     14.79: #####
     15.12: Customer 32047 departs
     15.12: ####
     15.55: Customer 32052 arrives
     15.55: #####
     15.64: Customer 32053 arrives
     15.64: ######
     16.12: Customer 32048 departs
     16.12: #####
     16.69: Customer 32054 arrives
     16.69: ######
     16.71: Customer 32055 arrives
     16.71: #######
     17.12: Customer 32049 departs
     17.12: ######
     18.12: Customer 32050 departs
     18.12: #####
     19.12: Customer 32051 departs
     19.12: ####
     19.49: Customer 32056 arrives
     19.49: #####
     20.12: Customer 32052 departs
     20.12: ####
     20.44: Customer 32057 arrives
     20.44: #####
     21.12: Customer 32053 departs
     21.12: ####
   

    392.49: Customer 32441 arrives
    392.49: #################
    392.76: Customer 32442 arrives
    392.76: ##################
    393.12: Customer 32425 departs
    393.12: #################
    394.12: Customer 32426 departs
    394.12: ################
    395.12: Customer 32427 departs
    395.12: ###############
    396.12: Customer 32428 departs
    396.12: ##############
    397.12: Customer 32429 departs
    397.12: #############
    398.12: Customer 32430 departs
    398.12: ############
    398.39: Customer 32443 arrives
    398.39: #############
    398.55: Customer 32444 arrives
    398.55: ##############
    399.12: Customer 32431 departs
    399.12: #############
    399.14: Customer 32445 arrives
    399.14: ##############
    399.68: Customer 32446 arrives
    399.68: ###############
    400.12: Customer 32432 departs
    400.12: ##############
    401.12: Customer 32433 departs
    401.12: #############
    402.12: Customer 32434 departs
    402.12: ############
  

    759.12: Customer 32791 departs
    759.12: #################################################
    759.20: Customer 32841 arrives
    759.20: ##################################################
    759.33: Customer 32842 arrives
    759.33: ###################################################
    759.86: Customer 32843 arrives
    759.86: ####################################################
    760.12: Customer 32792 departs
    760.12: ###################################################
    760.57: Customer 32844 arrives
    760.57: ####################################################
    761.12: Customer 32793 departs
    761.12: ###################################################
    762.12: Customer 32794 departs
    762.12: ##################################################
    762.36: Customer 32845 arrives
    762.36: ###################################################
    762.42: Customer 32846 arrives
    762.42: ####################################################
    762.81:

In [16]:
np.var(full_L_array)

1314.6066560073937